In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import scrublet as scr

In [ ]:
# Refer to dataset list for IDs
DATASET_ID = "TACA24GSE03"  # TODO

In [ ]:
OUTPUT_PATH = "../processed/qc2"
EXPORT_PATH = "../processed/export2"

MITO = set(open("../../../Mapping/mito.txt").read().splitlines())
RIBO = set(open("../../../Mapping/ribo.txt").read().splitlines())
with open("../../../Mapping/MappingGene.pkl", "rb") as f:
    MAPPING = pickle.load(f)


In [ ]:
# Load dataset
from scipy.io import mmread
import anndata as ad

count = mmread('../raw/count.mtx').tocsr()
cell = pd.read_csv('../raw/cell.tsv', sep = "\t", index_col = 0)
cell.index = list(cell['cell'])
cell.drop(columns = ['cell'], inplace = True)
gene = pd.read_csv('../raw/gene.txt', sep = "\t", header = None)
gene.rename(columns={0: "feature_name"}, inplace = True)
gene.index = gene.feature_name

adata = ad.AnnData(X=count.T, obs=cell, var=gene)

print("*Merged*", adata.shape)
SHAPES = [["RAW", *adata.shape], ]


In [ ]:
adata

In [ ]:
if adata.raw is not None:
    print("adata has the 'raw' attribute. Replacing data with 'raw'")
    adata = adata.raw.to_adata()
else:
    print("adata does not have the raw attribute.")

print("Please confirm that adata.X currently holds the raw counts!")
print(adata.X)


In [ ]:
# Dataset-level filtering if necessary, e.g., Human - Adult - Brain - 10x, etc.
# e.g., adata = adata[adata.obs["organism"] == "Homo sapiens"]
# e.g., adata = adata[adata.obs["assay"] == "10x 3' v3"]

print("No filtering")  # TODO

SHAPES.append(["DATASET_FILTERING", *adata.shape])


In [ ]:
# Replace with unique sample ID and remove all metadata. Use the same strategy from the metadata pipeline
# e.g., adata.obs["SampleID"] = adata.obs["donor_id"] 

adata.obs["SampleID"] = adata.obs["sample"]  # TODO
adata.obs = adata.obs[["SampleID"]]
print("# of samples:", adata.obs["SampleID"].nunique())


# Rename cell IDs to DatasetID_SampleID_CellID to avoid ID collision
adata.obs_names = DATASET_ID + "_" + adata.obs["SampleID"].astype(str) + "_" + adata.obs_names



In [ ]:
print(adata.var_names)

In [ ]:
# TODO change one of the below two variables to True if gene ID conversion to Ensembl is needed
is_entrez = False
is_symbol = True

if is_entrez:
    ensembl = [MAPPING["H2B"].get(MAPPING["E2H"].get(v, ""), "") for v in adata.var_names]
elif is_symbol:
    ensembl = [MAPPING["H2B"].get(MAPPING["S2H"].get(v, ""), "") for v in adata.var_names]
    
if is_entrez or is_symbol:
    new_id = []
    keep = []
    exist = set()
    for e in ensembl:
        if e and e not in exist:
            new_id.append(e)
            exist.add(e)
            keep.append(True)
        else:
            keep.append(False)
    adata = adata[:, keep]
    adata.var_names = new_id
    print(f"Mapped {len(exist)}/{len(keep)} genes")  # If the ratio of the mapped gene is too low, consider using "A2H" instead of "S2H"
else:
    print("No gene ID conversion is done")

SHAPES.append(["GENEID_MAPPING", *adata.shape])


In [ ]:
# Calculate QC metrics
adata.var["mt"] = [v in MITO for v in adata.var_names]
adata.var["rb"] = [v in RIBO for v in adata.var_names]

print("# Mito genes: ", adata.var["mt"].sum())
print("# Ribo genes: ", adata.var["rb"].sum())

sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "rb"], percent_top=None, log1p=False, inplace=True)


In [ ]:
# Let Barcode Rank Plot tell us whether ambient RNA removal needs to be done
nUMI = pd.DataFrame(adata.obs["total_counts"])
nUMI_sorted = nUMI.sort_values(by="total_counts", ascending=False)
nUMI_sorted["rank"] = range(1, len(nUMI_sorted) + 1)
plt.figure(figsize=(5, 4))
plt.loglog(nUMI_sorted["rank"], nUMI_sorted["total_counts"])
plt.title("Barcode Rank Plot")
plt.xlabel("Barcodes (ranked)")
plt.ylabel("UMI counts")
plt.savefig(f"{OUTPUT_PATH}/QC_Barcode_Rank_Plot.svg")
plt.show()


In [ ]:
# Plot before QC
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt", size = 0.5)
# plt.savefig(f"{OUTPUT_PATH}/QC_01.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_01.png")
plt.close()

sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", size = 0.5)
# plt.savefig(f"{OUTPUT_PATH}/QC_02.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_02.png")
plt.close()

sns.histplot(adata.obs["pct_counts_mt"], stat = "percent")
# plt.show()
# plt.savefig(f"{OUTPUT_PATH}/QC_03.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_03.png")
plt.close()

sns.histplot(adata.obs["pct_counts_rb"], stat = "percent")
# plt.show()
# plt.savefig(f"{OUTPUT_PATH}/QC_04.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_04_percent.png")
plt.close()


In [ ]:
# Universal QC
QCs = [
    adata.var["n_cells_by_counts"] >= 10,
    adata.obs["n_genes_by_counts"] >= 200,
    adata.obs["total_counts"] > 500,
]
QC_ROW_PASS = np.ones(adata.shape[0], dtype=bool) & QCs[1] & QCs[2]
QC_COL_PASS = np.ones(adata.shape[1], dtype=bool) & QCs[0]

SHAPES.append(["QC_1_PASS", QC_ROW_PASS.sum(), QC_COL_PASS.sum()])
SHAPES.append(["  MIN_CELLS>=10", "-", QCs[0].sum()])
SHAPES.append(["  MIN_GENES>=200", QCs[1].sum(), "-"])
SHAPES.append(["  TOTAL_COUNTS>500", QCs[2].sum(), "-"])

adata = adata[QC_ROW_PASS, :][:, QC_COL_PASS]


In [ ]:
SHAPES

In [ ]:
# Adaptive QC
QCs = []
for field in ("total_counts", "n_genes_by_counts"):
    values = adata.obs[field]
    q75, q25 = np.percentile(values, [75, 25])
    lower_co = q25 - 3 * (q75 - q25)
    upper_co = q75 + 3 * (q75 - q25)
    QCs.append((values > lower_co) & (values < upper_co))
    print(f"Threshold for {field}: {lower_co} < x < {upper_co}")

for field, threshold in (
        ("pct_counts_mt", 10),
        ("pct_counts_rb", 20)
):
    values = adata.obs[field]
    q75, q25 = np.percentile(values, [75, 25])
    upper_co = q75 + 3 * (q75 - q25)
    QCs.append(values < max(upper_co, threshold))
    print(f"Threshold for {field}: x < {max(upper_co, threshold)}")

QC_PASS = np.ones(adata.shape[0], dtype=bool) & QCs[0] & QCs[1] & QCs[2] & QCs[3]

SHAPES.append(["QC_2_PASS", QC_PASS.sum(), "-"])
SHAPES.append(["  TOTAL_COUNTS<3IQR", QCs[0].sum(), "-"])
SHAPES.append(["  N_GENES_BY_COUNTS<3IQR", QCs[1].sum(), "-"])
SHAPES.append(["  MT_PERCENT<3IQR", QCs[2].sum(), "-"])
SHAPES.append(["  RB_PERCENT<3IQR", QCs[3].sum(), "-"])

adata = adata[QC_PASS]


In [ ]:
SHAPES

In [ ]:
# Doublet detection
datasets = []
for sample in adata.obs["SampleID"].unique():
    sample_adata = adata[adata.obs["SampleID"] == sample].copy()
    expected = sample_adata.shape[0] / 1000 * 0.008
    print("Expected doublet rate for sample:", sample, expected)
    scrub = scr.Scrublet(sample_adata.X, expected_doublet_rate=expected)
    sample_adata.obs["doublet_scores"], sample_adata.obs["predicted_doublets"] = scrub.scrub_doublets(n_prin_comps=30)
    scrub.plot_histogram()
    plt.savefig(f"{OUTPUT_PATH}/{sample}_scrublet.svg")
    plt.close()
    datasets.append(sample_adata)


In [ ]:
# Doublet removal
adata = sc.concat(datasets, merge="unique")
adata = adata[adata.obs["predicted_doublets"] == False]
adata.obs.drop("predicted_doublets", axis=1, inplace=True)  # To avoid a potential TypeError

SHAPES.append(["DOUBLET", adata.shape[0], "-"])


In [ ]:
SHAPES

In [ ]:
# Plot after QC
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt", show=False)
# plt.savefig(f"{OUTPUT_PATH}/QC_11.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_11.png")
plt.close()

sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", show=False)
# plt.savefig(f"{OUTPUT_PATH}/QC_12.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_12.png")
plt.close()

sns.histplot(adata.obs["pct_counts_mt"])
# plt.show()
# plt.savefig(f"{OUTPUT_PATH}/QC_13.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_13.png")
plt.close()

sns.histplot(adata.obs["pct_counts_rb"])
# plt.show()
# plt.savefig(f"{OUTPUT_PATH}/QC_14.svg")
plt.savefig(f"{OUTPUT_PATH}/QC_14.png")
plt.close()


In [ ]:
# Output
SHAPES.append(["FINAL", *adata.shape])

with open(f"{OUTPUT_PATH}/QC.log", "w") as fo:
    for i, j, k in SHAPES:
        fo.write(f"{i}\t{j}\t{k}\n")

with open(f"{EXPORT_PATH}/genes.txt", "w") as fo:
    fo.write("\n".join(adata.var_names))

adata.write_h5ad(f"{EXPORT_PATH}/adata.h5ad")  # Count only


# Choose UMAP or tSNE depend on the original paper and data availability
# DIM_METHOD = "tSNE"  # TODO UMAP or tSNE

# X_dim = pd.DataFrame(adata.obsm["X_" + DIM_METHOD], index=adata.obs_names)
# X_dim.columns = [DIM_METHOD + "_1", DIM_METHOD + "_2"]
X_dim = adata.obs[["SampleID"]]
X_dim.to_csv(f"{EXPORT_PATH}/cell.tsv", sep="\t")

# Export normalized count for certain downstream analyses
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

from scipy import sparse, io

io.mmwrite(f"{EXPORT_PATH}/data.mtx", sparse.csr_matrix(adata.X))  # Normalized count, aka, "data"
